In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-13 21:52:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.98MB/s    in 0.2s    

2022-11-13 21:52:57 (4.98 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("videodvdETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
## Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

video_dvd_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_dvd_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

## Transform the data

### Count the number of records (rows) in the dataset.

In [ ]:
print(f'There are {video_dvd_df.count()} rows in this datatset.')

There are 5069140 rows in this datatset.


In [ ]:
# Drop null values, by default is any which means will drop any rows which has a NA
dropna_df = video_dvd_df.dropna(how='any')
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [ ]:
print(dropna_df.count())


5068421


In [ ]:
cleaned_video_dvd_df= dropna_df.dropDuplicates()
print(cleaned_video_dvd_df.count())

5068421


### Transform dataset to fit the tables

In [ ]:
cleaned_video_dvd_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Create review_id dataframe to match review_id table

review_id_df = cleaned_video_dvd_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005M0EAS6Q9U|   12077103|B00VUK505O|      69985058| 2015-07-21|
|R100AJQA6PHRS3|   34193335|B0000CG8HQ|     396111937| 2004-07-02|
|R100P4A9IY1O39|   26305542|B00GB0OV1C|     507977556| 2014-04-30|
|R100QA0LZ39480|    5874616|B000U1ZV6W|     427231257| 2015-01-08|
|R10119LK56IJO4|   50859438|B00005MP58|      22701133| 2013-01-14|
|R10122LZFC6IUV|   12325534|B004TJ1H3C|     961261650| 2015-01-16|
|R10168OWZ66QZO|   44723143|B00001PE4D|     104267530| 2001-03-21|
|R101DMT5D0HEWH|   52878930|B00LLQ2A2S|     735480698| 2015-01-04|
|R101E0NW9RIGX0|   35565191|B003QWVPT4|     339765617| 2014-10-20|
|R101OZ414P7ARI|    2624502|B00FXB8RRO|     737158350| 2014-12-23|
|R101R4FIHCUONZ|   42559842|B000PKG69O|     755015170| 2014-02-07|
|R101U37NQETQMP|   36403248|B00003CXWM|      67501200| 2013-01

In [ ]:
# Check the scheme of the review_id table
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# convert review_date from string to Date
from pyspark.sql.types import IntegerType,DateType
review_id_df =review_id_df.withColumn("review_date", review_id_df["review_date"].cast(DateType()))
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005M0EAS6Q9U|   12077103|B00VUK505O|      69985058| 2015-07-21|
|R100AJQA6PHRS3|   34193335|B0000CG8HQ|     396111937| 2004-07-02|
|R100P4A9IY1O39|   26305542|B00GB0OV1C|     507977556| 2014-04-30|
|R100QA0LZ39480|    5874616|B000U1ZV6W|     427231257| 2015-01-08|
|R10119LK56IJO4|   50859438|B00005MP58|      22701133| 2013-01-14|
|R10122LZFC6IUV|   12325534|B004TJ1H3C|     961261650| 2015-01-16|
|R10168OWZ66QZO|   44723143|B00001PE4D|     104267530| 2001-03-21|
|R101DMT5D0HEWH|   52878930|B00LLQ2A2S|     735480698| 2015-01-04|
|R101E0NW9RIGX0|   35565191|B003QWVPT4|     339765617| 2014-10-20|
|R101OZ414P7ARI|    2624502|B00FXB8RRO|     737158350| 2014-12-23|
|R101R4FIHCUONZ|   42559842|B000PKG69O|     755015170| 2014-02-07|
|R101U37NQETQMP|   36403248|B00003CXWM|      67501200| 2013-01

In [ ]:
# Create products dataframe to match products table. 
products_df = cleaned_video_dvd_df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001DE29SS|Chaplin (15th Ann...|
|B00ZJ2GL6G|Jurassic World 3D...|
|B00T5DM876|Pretty Little Lia...|
|B004VW4V76|         Lebanon, PA|
|B009D4SFEC|Murdoch Mysteries...|
|B000GG4Y5K|Numb3rs - The Com...|
|B00383XZP8|TCM Spotlight: Ch...|
|B00I9MS86O|Breaking Bad: The...|
|B00005K3O4|Killer Klowns Fro...|
|B00BD6KETC|     Psych: Season 7|
|B00WHZZBTQ|Heaven Adores You...|
|B000092Q5C|Remo Williams - T...|
|B004AXPWH4|10 MINUTESOLUTION...|
|B00FEP9PQG|Boardwalk Empire:...|
|B0002F6BJW|Fractured Flicker...|
|B0007LXP7W|The A-Team - Seas...|
|B005ET4NHI|  Where The Boys Are|
|B005EBOSO4|Muscle Energy Tec...|
|B003V5CTKU|Black Label Socie...|
|B000087F6L|      Fahrenheit 451|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products_df.count()

297902

In [ ]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# Create customers dataframe to match customers table
customers_df = cleaned_video_dvd_df.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   25498831|             2|
|   47490390|             3|
|   32702184|            22|
|   10206054|            16|
|   42780745|             1|
|   23105709|             2|
|   36176703|            35|
|   28707214|             3|
|   45286909|           195|
|   32142680|            77|
|   50927426|             1|
|   19540657|             2|
|   15422036|             1|
|   45763189|             9|
|   38851882|            92|
|   10704478|             1|
|   35523976|            21|
|   14564825|            13|
|   35390414|             1|
|   49243158|           274|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Check the customers table schema 
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
# Change customer_count to integer type
from pyspark.sql.types import IntegerType
customers_df =customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
# Create vine_table dataframe to match vine_table table
vine_table_df = cleaned_video_dvd_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005M0EAS6Q9U|          5|            0|          1|   N|
|R100AJQA6PHRS3|          1|           21|         29|   N|
|R100P4A9IY1O39|          1|            0|         10|   N|
|R100QA0LZ39480|          5|            0|          0|   N|
|R10119LK56IJO4|          5|            0|          0|   N|
|R10122LZFC6IUV|          5|            0|          0|   N|
|R10168OWZ66QZO|          5|            1|          1|   N|
|R101DMT5D0HEWH|          5|            1|          1|   N|
|R101E0NW9RIGX0|          4|            0|          0|   N|
|R101OZ414P7ARI|          5|            0|          0|   N|
|R101R4FIHCUONZ|          3|            1|          1|   N|
|R101U37NQETQMP|          5|            0|          0|   N|
|R101UN1Y26WGQ3|          4|            2|          6|   N|
|R1027YFOX85498|          4|            

In [ ]:
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# Load the Data

Postgres Setup

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://videodb.rds.amazonaws.com:5432/videodvd"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to review_id table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS

products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)